In [94]:
import numpy as np
import pandas as pd
import fancyimpute
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegressionCV
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
%matplotlib inline
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'fancyimpute'

In [95]:
# Examine head of data
df = pd.read_csv("Final_P_3.csv")
df.head(100)

,Year,Player,Position,Age,School,Height in Feet,Weight In Pounds,40 Yard Dash,Vertical,Bench Reps,Broad Jump,3Cone,Shuttle,Round,Team,Conference
0,2020,Jedrick Wills,OL,20,Alabama,6.40,312,5.05,34.5,0,113,0.00,4.84,1,Cleveland Browns,SEC
1,2020,Jerry Jeudy,WR,20,Alabama,6.10,193,4.45,35.0,0,120,0.00,4.53,1,Denver Broncos,SEC
2,2020,Henry Ruggs III,WR,21,Alabama,5.11,188,4.27,42.0,0,131,0.00,0.00,1,Las Vegas Raiders,SEC
3,2020,Tua Tagovailoa,QB,22,Alabama,6.10,217,0.00,0.0,0,0,0.00,0.00,1,Miami Dolphins,SEC
4,2020,Xavier McKinney,S,0,Alabama,6.10,201,4.63,36.0,19,122,0.00,0.00,2,New York Giants,SEC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2020,Lavert Hill,CB,0,Michigan,5.10,190,0.00,0.0,21,0,0.00,0.00,0,0,Big 10
96,2020,Josiah Scott,CB,0,Michigan State,5.90,185,4.42,0.0,17,0,0.00,0.00,4,Jacksonville Jaguars,Big 10
97,2020,Kenny Willekes,DL,0,Michigan State,6.30,264,4.87,32.5,32,119,7.39,0.00,7,Minnesota Vikings,Big 10
98,2020,Brian Lewerke,QB,0,Michigan State,6.20,213,4.95,31.0,0,113,7.14,4.40,0,0,Big 10


In [96]:
df.groupby("Position").size()

Position
CB    19
DL    25
K      1
LB    25
LS     1
OL    34
P      6
QB    11
RB    17
S     11
TE    15
WR    35
dtype: int64

In [97]:
df[df.Position=="K"]

,Year,Player,Position,Age,School,Height in Feet,Weight In Pounds,40 Yard Dash,Vertical,Bench Reps,Broad Jump,3Cone,Shuttle,Round,Team,Conference
180,2020,JJ Molson,K,0,UCLA,5.11,182,0.0,0.0,0,0,0.0,0.0,0,0,Pac 12


In [98]:
# JJ Molson will play mostly as a K.
df.at[180, "Position"] = "K"

In [99]:
df[df.Position=="LS"]

,Year,Player,Position,Age,School,Height in Feet,Weight In Pounds,40 Yard Dash,Vertical,Bench Reps,Broad Jump,3Cone,Shuttle,Round,Team,Conference
65,2020,Steven Wirtel,LS,0,Iowa State,6.4,235,4.76,26.0,0,120,7.12,4.28,1,0,Big 10


In [100]:
# Steven Wirtel is a LS.
df.at[65, "Position"] = "LS"

In [101]:
df.loc[df.Position=="P", "Position"] = "P"

In [102]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [103]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [104]:
classifier.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [105]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 1.0
Testing Data Score: 1.0


In [106]:
# All of our "hybrid" positions have been dealt with. Let's now examine preliminary trends.
round_info = pd.crosstab(index=df["Position"], columns=df["Round"], margins=True) 
round_info / round_info.loc["All"]

Round,0,1,2,3,4,5,6,7,All
Position,,,,,,,,,
CB,0.066667,0.090909,0.066667,0.083333,0.260870,0.047619,0.000000,0.176471,0.095
DL,0.116667,0.090909,0.000000,0.166667,0.086957,0.238095,0.111111,0.176471,0.125
K,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005
LB,0.083333,0.136364,0.133333,0.208333,0.086957,0.190476,0.055556,0.176471,0.125
LS,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005
OL,0.116667,0.272727,0.066667,0.291667,0.130435,0.190476,0.333333,0.000000,0.170
P,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.058824,0.030
QB,0.050000,0.136364,0.066667,0.000000,0.043478,0.000000,0.055556,0.117647,0.055
RB,0.166667,0.000000,0.133333,0.083333,0.130435,0.000000,0.000000,0.000000,0.085


In [107]:
round_info = pd.crosstab(index=df["Position"], columns=df["Round"], margins=True) 
round_info.div(round_info["All"], axis=0)

Round,0,1,2,3,4,5,6,7,All
Position,,,,,,,,,
CB,0.210526,0.105263,0.052632,0.105263,0.315789,0.052632,0.000000,0.157895,1.0
DL,0.280000,0.080000,0.000000,0.160000,0.080000,0.200000,0.080000,0.120000,1.0
K,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
LB,0.200000,0.120000,0.080000,0.200000,0.080000,0.160000,0.040000,0.120000,1.0
LS,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
OL,0.205882,0.176471,0.029412,0.205882,0.088235,0.117647,0.176471,0.000000,1.0
P,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.166667,1.0
QB,0.272727,0.272727,0.090909,0.000000,0.090909,0.000000,0.090909,0.181818,1.0
RB,0.588235,0.000000,0.117647,0.117647,0.176471,0.000000,0.000000,0.000000,1.0


In [109]:
df.loc[df.Round != 1, "Round"] = 0

In [110]:
all_X = df.drop(["Player", "Round", "School", "Conference"], 1)
all_X = pd.get_dummies(all_X)

train_X = all_X[(all_X.Year != 2020)].drop("Year", 1)
test_X = all_X[all_X.Year == 2020].drop("Year", 1)
train_y = df[(df.Year != 2020)].Round
test_y = df[df.Year == 2020].Round

In [111]:
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
0,0,0
1,1,1
2,1,1
3,0,0
4,0,0
5,1,1
6,0,0
7,0,0
8,0,0
9,1,1
